<img src="https://companieslogo.com/img/orig/MMYT-ca98a9f1.png?t=1602681214" width="70" height="90">

# `Make My Trip` : Case Study

A manager at a travel app has come up with some questions and asks the __Data Analyst in the company__ to fetch the answers for the same:

### Questions: 

1. Get total counts of ___segment-wise___: 
    - `Unique users`
    - `Users who booked flight in the month of April`


2. Get all users who started using the app by booking `Hotel` first


3. No. of days between first booking and last booking for all users


4. Total `Flights` and `Hotel` bookings by each segment

-----

# Import libraries & Connections

In [1]:
import os
import pandas as pd
import sqlite3 as s3

con = s3.connect("database.db")

# Functions to reproduce `DDL` and `DML` steps

In [2]:
def create_table(query, con=con):
    for i in table.split(';'):
        con.execute(i.strip())

def query(query, con=con):
    return pd.read_sql(query, con)

def delete_db(db_name):
    try:
        os.remove(db_name)
        os.remove(db_name+'-journal')
    except:
        pass

In [3]:
# Creating the tables - Bookings & Users

table = """CREATE TABLE booking_table(
   Booking_id       VARCHAR(3) NOT NULL 
  ,Booking_date     date NOT NULL
  ,User_id          VARCHAR(2) NOT NULL
  ,Line_of_business VARCHAR(6) NOT NULL
);
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b1','2022-03-23','u1','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b2','2022-03-27','u2','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b3','2022-03-28','u1','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b4','2022-03-31','u4','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b5','2022-04-02','u1','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b6','2022-04-02','u2','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b7','2022-04-06','u5','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b8','2022-04-06','u6','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b9','2022-04-06','u2','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b10','2022-04-10','u1','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b11','2022-04-12','u4','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b12','2022-04-16','u1','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b13','2022-04-19','u2','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b14','2022-04-20','u5','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b15','2022-04-22','u6','Flight');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b16','2022-04-26','u4','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b17','2022-04-28','u2','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b18','2022-04-30','u1','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b19','2022-05-04','u4','Hotel');
INSERT INTO booking_table(Booking_id,Booking_date,User_id,Line_of_business) VALUES ('b20','2022-05-06','u1','Flight');
;
CREATE TABLE user_table(
   User_id VARCHAR(3) NOT NULL
  ,Segment VARCHAR(2) NOT NULL
);
INSERT INTO user_table(User_id,Segment) VALUES ('u1','s1');
INSERT INTO user_table(User_id,Segment) VALUES ('u2','s1');
INSERT INTO user_table(User_id,Segment) VALUES ('u3','s1');
INSERT INTO user_table(User_id,Segment) VALUES ('u4','s2');
INSERT INTO user_table(User_id,Segment) VALUES ('u5','s2');
INSERT INTO user_table(User_id,Segment) VALUES ('u6','s3');
INSERT INTO user_table(User_id,Segment) VALUES ('u7','s3');
INSERT INTO user_table(User_id,Segment) VALUES ('u8','s3');
INSERT INTO user_table(User_id,Segment) VALUES ('u9','s3');
INSERT INTO user_table(User_id,Segment) VALUES ('u10','s3');
"""

create_table(table)

------

### 1. Segment Wise Information

In [10]:
q = """

SELECT 
    u.segment, 
    count(distinct u.user_id) as Users,
    count(distinct CASE WHEN b.Line_of_business = 'Flight' AND 
                             booking_date BETWEEN '2022-04-01' AND '2022-04-30'
                        THEN b.user_id END) as Users_on_flight
FROM user_table u
    left Join booking_table b ON u.user_id = b.user_id
Group By u.segment
"""

query(q)

,Segment,Users,Users_on_flight
0,s1,3,2
1,s2,2,2
2,s3,5,1


------

### 2. Users whose first booking is `Hotel`

###### method 1

In [6]:
q = """
SELECT 
    user_id 
FROM (SELECT *, 
             RANK() OVER(PARTITION BY user_id ORDER BY booking_date) AS rn 
      FROM booking_table)
      
WHERE rn=1 AND line_of_business = 'Hotel'
"""

query(q)

,User_id
0,u6


###### method 2

In [8]:
q = """
SELECT 
    DISTINCT user_id 
FROM (SELECT *, 
             FIRST_VALUE(line_of_business) OVER(PARTITION BY user_id ORDER BY booking_date) AS first_booking 
      FROM booking_table)
WHERE first_booking = 'Hotel'
"""

query(q)

,User_id
0,u6


-------

### 3. No. of Days between first and last booking of each user

In [13]:
q = """

SELECT 
    user_id, 
    MIN(booking_date) as first_date, 
    MAX(booking_date) as last_date, 
    (JULIANDAY(MAX(booking_date))- JULIANDAY(MIN(booking_date))) as number_of_days
FROM booking_table
GROUP BY user_id

"""

query(q)

,User_id,first_date,last_date,number_of_days
0,u1,2022-03-23,2022-05-06,44.0
1,u2,2022-03-27,2022-04-28,32.0
2,u4,2022-03-31,2022-05-04,34.0
3,u5,2022-04-06,2022-04-20,14.0
4,u6,2022-04-06,2022-04-22,16.0


------

### 4. Total no. of `flights` and `hotel` bookings in each segment

In [14]:
q = """

SELECT 
    u.segment,
    COUNT(CASE WHEN line_of_business = "Flight" THEN Booking_id END) AS Flight_Bookings,
    COUNT(CASE WHEN line_of_business = "Hotel" THEN Booking_id END) AS Hotel_Bookings
FROM user_table u
LEFT JOIN Booking_table b ON b.user_id = u.user_id
GROUP BY u.segment

"""

query(q)

,Segment,Flight_Bookings,Hotel_Bookings
0,s1,8,4
1,s2,3,3
2,s3,1,1


---